In [9]:
from web_summarizer import WebSummarizer, ChatGPTSummarizer, OllamaSummarizer
from IPython.display import Markdown, display

In [10]:
# url = "https://www.prosigmaka.com"
# url = "https://www.prodemy.id"
url = "http://www.detik.com"
# url = "http://www.tempo.co"
# url = "https://www.kompas.com/"

In [12]:
# ChatGPT Summarizer
llm_summarizer = ChatGPTSummarizer()
web_summarizer = WebSummarizer(url, llm_summarizer)
summarized = web_summarizer.summarize()
display(Markdown(summarized))

# Summary of Detikcom

**Website Title:** Detikcom - Informasi Berita Terkini dan Terbaru Hari Ini

Detikcom is a news website providing up-to-date and reliable information on a variety of topics including current events, finance, technology, sports, and more. The site features articles categorized into sections like detikNews, detikFinance, detikSport, and others, catering to diverse interests.

## Latest News Highlights
- **Isa Zega Detained**: Following a case related to Information and Electronic Transactions (ITE), Nikita Mirzani was briefly a witness.
- **SPBU Incident**: A 'cowboy' style incident where an individual threatened to shoot gas station personnel has resulted in their detention.
- **Traffic Update**: The elevated toll road MBZ from Km 17-22 towards Cikampek is experiencing congestion due to a vehicle malfunction.
- **Glenn Alinskie's Health**: The actor has undergone surgery for hypertrophy and is currently recovering.
- **Tea Consumption in the UK**: A surprising statistic reveals that the UK consumes 53 million cups of tea every morning.

## Other Noteworthy Updates
- **Donald Trump and Kim Jong Un**: Discussions about their relationship reveal new dynamics in their latest interactions.
- **Recent Sports News**: Kyle Walker has officially joined AC Milan and there are upcoming matches in the Indonesia Masters 2025.

Detikcom also includes videos, photo stories, and trending topics to keep readers engaged with the latest happenings.

In [11]:
# Ollama Summarizer
llm_summarizer = OllamaSummarizer()
web_summarizer = WebSummarizer(url, llm_summarizer)
summarized = web_summarizer.summarize()
display(Markdown(summarized))

Terdapat 5 berita terpopuler di situs detikNews yang disebutkan di atas. Berdasarkan judul-judul berita tersebut, dapat disimpulkan bahwa ada beberapa topik yang dipaparkan dalam berita-berita tersebut. Berikut adalah penjelasan singkat tentang setiap berita:

1. "Misteri Pemicu 2 Pria Sekamar Kosan Saling Tusuk di Priok" - Berita ini membahas tentang kejadian yang melibatkan dua pria yang menyerang dan merahmati satu sama lain di sebuah kosan di Priok, Jakarta Selatan. Kedua pria tersebut kemudian ditembak oleh petugas keamanan kosan dan dibawa ke polisi.

2. "Prabowo Mampir ke Toko Buku Langganan di Sela Kunker ke India" - Berita ini meliputi kunjungan Presiden Joko Widodo (Jokowi) kepada India, di mana dia membatalkan kunjungannya ke Pakistan dan mendarat di Bandar Udara Internasional Bengalurunya, India. Selama kunjungan tersebut, Prabowo juga mengunjungi toko buku langganan di Sela Kunker, kota yang terletak di negara bagian Karnataka, India.

3. "Ulah 'Koboi' Ancam Tembak Petugas SPBU Berujung Masuk Tahanan" - Berita ini membahas tentang kejadian yang melibatkan seorang ulah yang menyerang dan merahmati petugas SPBU di sebuah kawasan perumahan. Ulah tersebut kemudian ditembak oleh petugas keamanan dan dibawa ke polisi.

4. "3 Striker Muda Australia Siap Debut, Jadi Ancaman Indonesia" - Berita ini meliputi perspektif tentang kehadiran tiga striker muda dari Australia yang siap debut di pesta Liga, di mana mereka dapat menjadi ancaman bagi tim-tim lokal.

5. "Guardiola Jelaskan Alasan Man City Boyong Marmoush" - Berita ini membahas tentang keputusan pemain Tim Nasional Indonesia untuk bergabung dengan klub PFA Inggris Manchester City dan mengapa mereka memilih klub tersebut.